<a href="https://colab.research.google.com/github/shesshirley/Goldman-Analysis/blob/main/Goldman_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/Shared drives/datathon'

Mounted at /content/drive
/content/drive/Shared drives/datathon


In [ ]:
import pandas as pd
import numpy as np

# Data Collection

In [ ]:
#using get query to save datasets and read into dataframes
stock_df = pd.read_csv("stock_history.csv")
env_df = pd.read_csv("env_data.csv")
company_df = pd.read_csv("company_profile.csv")

# Data Analysis
### Data Pre-Processing

In [ ]:
#drop the useless column and keep the valuable feature for later predicting, pre-process the date column and filter out the date before 1980
stock_df1 = stock_df.copy()
stock_df1['YEAR'] = pd.to_datetime(stock_df1['DATE']).dt.year
stock_df1['MONTH'] = pd.to_datetime(stock_df1['DATE']).dt.month
stock_df1['DAY'] = pd.to_datetime(stock_df1['DATE']).dt.day
stock_df1['YEAR-MONTH'] = pd.to_datetime(stock_df1['DATE']).dt.to_period('M')
stock_df1 = stock_df1[stock_df1["YEAR"]>=1980]
stock_df1.head()

,ID,SYMBOL,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,ADJCLOSE,YEAR,MONTH,DAY,YEAR-MONTH
9,NaN,BB,2020-08-07,4.97,5.16,4.81,4.84,5361500.0,4.84,2020,8,7,2020-08
10,NaN,BB,2020-06-03,5.38,5.43,5.16,5.21,5690900.0,5.21,2020,6,3,2020-06
11,NaN,BB,2020-01-23,6.60,6.67,6.52,6.66,2659800.0,6.66,2020,1,23,2020-01
12,NaN,BB,2019-10-23,5.12,5.26,5.12,5.21,4293500.0,5.21,2019,10,23,2019-10
13,NaN,BB,2019-03-06,8.72,9.15,8.62,9.08,10410200.0,9.08,2019,3,6,2019-03


In [ ]:
open_date = stock_df1.groupby('YEAR-MONTH')['DAY'].min().reset_index()
close_date = stock_df1.groupby('YEAR-MONTH')['DAY'].max().reset_index()
stock_df1["MONTH_OPEN"] = stock_df1.merge(open_date, on=['YEAR-MONTH','DAY'],how='inner')['OPEN']
stock_df1["MONTH_CLOSE"] = stock_df1.merge(close_date, on=['YEAR-MONTH','DAY'],how='inner')['CLOSE']
stock_df1.head()

,ID,SYMBOL,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,ADJCLOSE,YEAR,MONTH,DAY,YEAR-MONTH,MONTH_OPEN,MONTH_CLOSE
9,NaN,BB,2020-08-07,4.97,5.16,4.81,4.84,5361500.0,4.84,2020,8,7,2020-08,12.9594,4.1786
10,NaN,BB,2020-06-03,5.38,5.43,5.16,5.21,5690900.0,5.21,2020,6,3,2020-06,26.4700,44.0739
11,NaN,BB,2020-01-23,6.60,6.67,6.52,6.66,2659800.0,6.66,2020,1,23,2020-01,32.9796,9.9753
12,NaN,BB,2019-10-23,5.12,5.26,5.12,5.21,4293500.0,5.21,2019,10,23,2019-10,19.7000,20.1400
13,NaN,BB,2019-03-06,8.72,9.15,8.62,9.08,10410200.0,9.08,2019,3,6,2019-03,10.0000,2.0586


In [ ]:
env_df1 = env_df.copy()
env_df1 = env_df1[['Country ISO Code', \
       'Indicator', 'Description','Year', 'Unit','Value']]
env_df1 = env_df1[env_df1["Year"]>=1980]
env_df1.head(2)

,Country ISO Code,Indicator,Description,Year,Unit,Value
10,AE,CO2 emissions,Carbon dioxide emissions are those stemming fr...,1980,Mton CO2,41.11924
11,AE,CO2 emissions,Carbon dioxide emissions are those stemming fr...,1981,Mton CO2,38.04811


In [ ]:
env_df2 = env_df1.groupby(['Year', 'Indicator']).Value.sum().unstack()
env_df2.head()

Indicator,Agricultural methane emissions,Agricultural methane emissions (%),Agricultural nitrous oxide emissions,Agricultural nitrous oxide emissions (%),CO2 emissions,CO2 emissions from bunker fuels,CO2 emissions from cement production,CO2 emissions from electricity production (%),CO2 emissions from fossil-fuels,CO2 emissions from gas flaring,CO2 emissions from gas fuel consumption,CO2 emissions from gaseous fuel consumption,CO2 emissions from gaseous fuel consumption (%),CO2 emissions from liquid fuel consumption,CO2 emissions from liquid fuel consumption (%),CO2 emissions from manufacturing industries (%),CO2 emissions from other sectors (%),CO2 emissions from solid fuel consumption,CO2 emissions from solid fuel consumption (%),CO2 emissions intensity,CO2 emissions per capita,CO2 emissions: residential buildings (%),CO2 emissions: transport (%),CO2 intensity,Consumption chlorofluorocarbons,Consumption of hydrochlorofluorocarbons,Consumption of methyl bromide,Consumption of ozone-depleting substances,Energy related methane emissions,Marine protected areas,Methane emissions,Methane emissions in energy sector,Nitrous oxide emissions,Nitrous oxide emissions in energy,Other greenhouse gas emissions,PM2.5 air pollution,Quantity of municipal waste collected,Terrestrial and marine protected areas,Terrestrial protected areas,Threatened bird species,Threatened fish species,Threatened mammal species,Threatened plant species,Use of nitrogen,Use of phosphate,Use of potash
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1980,3.071149e+06,9966.76698,1.568752e+06,12254.43371,18091.80037,104245.0,84346.0,3181.09850,3583001.0,79733.0,491804.0,2554164.509,977.87869,9547300.083,11820.10797,2723.16759,589.60718,7551732.155,1977.64737,NaN,1208.74617,1285.18072,3218.61769,254.81663,NaN,NaN,NaN,NaN,4120.71176,NaN,5.991702e+06,1.778504e+06,2.704464e+06,190644.57852,4.925495e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1981,3.026422e+06,9960.17790,1.521332e+06,12311.12862,17787.48137,95737.0,85016.0,3255.65835,3496746.0,58246.0,491038.0,2599209.937,1052.26974,9132716.338,11859.32974,2715.75154,578.42577,7498272.234,1976.44079,NaN,1134.41685,1277.78739,3271.11069,243.63632,NaN,NaN,NaN,NaN,4152.68251,NaN,5.851793e+06,1.727046e+06,2.608747e+06,190406.15223,4.472236e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1982,3.069972e+06,9915.27366,1.597245e+06,12316.97682,17615.40333,85814.0,86407.0,3329.35908,3470982.0,58115.0,465682.0,2548576.001,1045.57262,8953865.159,11795.81588,2647.19429,568.63327,7670003.584,2058.19139,NaN,1105.34378,1261.85695,3293.67046,237.74315,NaN,NaN,NaN,NaN,4096.73010,NaN,6.269163e+06,1.762952e+06,2.778074e+06,192603.16155,6.095739e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1983,3.061073e+06,9794.18920,1.612970e+06,12256.46280,17777.75733,83353.0,89872.0,3398.21203,3516130.0,52269.0,460276.0,2587578.213,1099.51030,8906376.642,11715.98979,2603.60537,593.46652,7926593.963,2135.02805,NaN,1080.44134,1217.94960,3286.09157,238.43865,NaN,NaN,NaN,NaN,4140.55462,NaN,6.120151e+06,1.793234e+06,2.753106e+06,195212.48911,5.635440e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984,3.033687e+06,9790.46621,1.601254e+06,12318.89175,18388.17896,83882.0,93603.0,3439.16864,3649042.0,45179.0,497785.0,2801881.360,1191.28291,8953451.622,11557.53199,2579.15563,563.29662,8283522.397,2230.03724,NaN,1117.83456,1248.28421,3267.00953,233.25241,NaN,NaN,NaN,NaN,4252.27703,NaN,5.966796e+06,1.829739e+06,2.667446e+06,199030.93124,4.237173e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
company_df1 = company_df.copy()
company_df1 = company_df1[['ID', 'SYMBOL', 'PRICE', 'BETA', 'VOLAVG', 'MKTCAP', 'LASTDIV', 'RANGE',
       'CHANGES', 'COMPANYNAME', 'INDUSTRY', 'SECTOR', 'DCFDIFF', 'DCF']]
company_df1.head(2)

,ID,SYMBOL,PRICE,BETA,VOLAVG,MKTCAP,LASTDIV,RANGE,CHANGES,COMPANYNAME,INDUSTRY,SECTOR,DCFDIFF,DCF
0,4276,GDP,8.61,2.26205,23768.0,108966432.0,0.00,2.395-11.71,0.27,Goodrich Petroleum Corp,Oil & Gas E&P,Energy,632.38,8.4445
1,4277,RFIL,4.47,0.92337,40537.0,43678692.0,0.08,3.31-7.38,0.01,RF Industries Ltd,Electrical Equipment & Parts,Industrials,NaN,5.7325


# Data Modeling
### Data Pre-Processing

In [ ]:
env_df3 = env_df2.copy()
for i in env_df3.columns:
  if env_df3[i].isna().sum() > 10:
   env_df3 = env_df3.drop(i,axis=1)
env_df3 = env_df3.reset_index()
env_df3.head(2)

Indicator,Year,Agricultural methane emissions,Agricultural nitrous oxide emissions,CO2 emissions,CO2 emissions from bunker fuels,CO2 emissions from cement production,CO2 emissions from electricity production (%),CO2 emissions from fossil-fuels,CO2 emissions from gas flaring,CO2 emissions from gas fuel consumption,CO2 emissions from gaseous fuel consumption,CO2 emissions from gaseous fuel consumption (%),CO2 emissions from liquid fuel consumption,CO2 emissions from liquid fuel consumption (%),CO2 emissions from manufacturing industries (%),CO2 emissions from other sectors (%),CO2 emissions from solid fuel consumption,CO2 emissions from solid fuel consumption (%),CO2 emissions intensity,CO2 emissions per capita,CO2 emissions: residential buildings (%),CO2 emissions: transport (%),CO2 intensity,Methane emissions,Methane emissions in energy sector,Nitrous oxide emissions,Nitrous oxide emissions in energy,Other greenhouse gas emissions
0,1980,3.071149e+06,1.568752e+06,18091.80037,104245.0,84346.0,3181.09850,3583001.0,79733.0,491804.0,2554164.509,977.87869,9547300.083,11820.10797,2723.16759,589.60718,7551732.155,1977.64737,NaN,1208.74617,1285.18072,3218.61769,254.81663,5.991702e+06,1.778504e+06,2.704464e+06,190644.57852,4.925495e+06
1,1981,3.026422e+06,1.521332e+06,17787.48137,95737.0,85016.0,3255.65835,3496746.0,58246.0,491038.0,2599209.937,1052.26974,9132716.338,11859.32974,2715.75154,578.42577,7498272.234,1976.44079,NaN,1134.41685,1277.78739,3271.11069,243.63632,5.851793e+06,1.727046e+06,2.608747e+06,190406.15223,4.472236e+06


In [ ]:
company_df2=company_df1[company_df1['INDUSTRY']=='Oil & Gas Integrated']
company_df2.head(2)

,ID,SYMBOL,PRICE,BETA,VOLAVG,MKTCAP,LASTDIV,RANGE,CHANGES,COMPANYNAME,INDUSTRY,SECTOR,DCFDIFF,DCF
812,5309,AE,19.8800,1.02984,6104.0,84336520.0,0.960,15.16-39.7,-0.1200,Adams Resources & Energy Inc,Oil & Gas Integrated,Energy,NaN,46.6936
1570,6357,ECT,0.4002,2.03408,31428.0,7045521.0,0.141,0.227-1.21,-0.0117,ECA Marcellus Trust I,Oil & Gas Integrated,Energy,NaN,0.0000


In [ ]:
stock_df2 = stock_df1.merge(company_df2, on='SYMBOL', how='inner')
stock_df2.head(2)

,ID_x,SYMBOL,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,ADJCLOSE,YEAR,MONTH,DAY,YEAR-MONTH,MONTH_OPEN,MONTH_CLOSE,ID_y,PRICE,BETA,VOLAVG,MKTCAP,LASTDIV,RANGE,CHANGES,COMPANYNAME,INDUSTRY,SECTOR,DCFDIFF,DCF
0,NaN,SU.TO,2020-05-08,23.54,24.13,23.44,23.83,9544100.0,23.83,2020,5,8,2020-05,18.91,1.6577,11037,16.04,1.6981,8313579.0,2.446341e+10,1.935,14.02-45.12,0.09,Suncor Energy Inc,Oil & Gas Integrated,Energy,NaN,NaN
1,NaN,SU.TO,2019-11-08,42.30,42.75,42.16,42.66,3158600.0,41.69,2019,11,8,2019-11,11.20,3.1600,11037,16.04,1.6981,8313579.0,2.446341e+10,1.935,14.02-45.12,0.09,Suncor Energy Inc,Oil & Gas Integrated,Energy,NaN,NaN


In [ ]:
stock_df3 = stock_df2.groupby('YEAR-MONTH')[['SYMBOL', 'MONTH_OPEN', 'VOLUME', 'YEAR', 'MONTH',
       'MKTCAP', 'LASTDIV', 'RANGE', 'CHANGES','MONTH_CLOSE']].mean().reset_index().rename(columns={'YEAR':'Year'})
stock_df3.head(2)

,YEAR-MONTH,MONTH_OPEN,VOLUME,Year,MONTH,MKTCAP,LASTDIV,CHANGES,MONTH_CLOSE
0,1980-01,30.799436,1.867701e+06,1980,1,6.770087e+10,2.666,0.64868,19.682982
1,1980-02,16.897300,2.082418e+06,1980,2,6.770087e+10,2.666,0.64868,6.720550


In [ ]:
stock_df3["TIME"] = (stock_df3["Year"]-1980)*12 +stock_df3["MONTH"]

In [ ]:
df = stock_df3.merge(env_df3, on='Year')
df = df.drop("Year",axis=1)
df = df.drop("YEAR-MONTH",axis=1)
df = df.drop("MONTH",axis=1)
df.head(2)

,MONTH_OPEN,VOLUME,MKTCAP,LASTDIV,CHANGES,MONTH_CLOSE,TIME,Agricultural methane emissions,Agricultural nitrous oxide emissions,CO2 emissions,CO2 emissions from bunker fuels,CO2 emissions from cement production,CO2 emissions from electricity production (%),CO2 emissions from fossil-fuels,CO2 emissions from gas flaring,CO2 emissions from gas fuel consumption,CO2 emissions from gaseous fuel consumption,CO2 emissions from gaseous fuel consumption (%),CO2 emissions from liquid fuel consumption,CO2 emissions from liquid fuel consumption (%),CO2 emissions from manufacturing industries (%),CO2 emissions from other sectors (%),CO2 emissions from solid fuel consumption,CO2 emissions from solid fuel consumption (%),CO2 emissions intensity,CO2 emissions per capita,CO2 emissions: residential buildings (%),CO2 emissions: transport (%),CO2 intensity,Methane emissions,Methane emissions in energy sector,Nitrous oxide emissions,Nitrous oxide emissions in energy,Other greenhouse gas emissions
0,30.799436,1.867701e+06,6.770087e+10,2.666,0.64868,19.682982,1,3.071149e+06,1.568752e+06,18091.80037,104245.0,84346.0,3181.0985,3583001.0,79733.0,491804.0,2554164.509,977.87869,9547300.083,11820.10797,2723.16759,589.60718,7551732.155,1977.64737,NaN,1208.74617,1285.18072,3218.61769,254.81663,5.991702e+06,1.778504e+06,2.704464e+06,190644.57852,4.925495e+06
1,16.897300,2.082418e+06,6.770087e+10,2.666,0.64868,6.720550,2,3.071149e+06,1.568752e+06,18091.80037,104245.0,84346.0,3181.0985,3583001.0,79733.0,491804.0,2554164.509,977.87869,9547300.083,11820.10797,2723.16759,589.60718,7551732.155,1977.64737,NaN,1208.74617,1285.18072,3218.61769,254.81663,5.991702e+06,1.778504e+06,2.704464e+06,190644.57852,4.925495e+06


In [ ]:
df.isna().sum()

MONTH_OPEN                                           0
VOLUME                                               0
MKTCAP                                               0
LASTDIV                                              0
CHANGES                                              0
MONTH_CLOSE                                          0
TIME                                                 0
Agricultural methane emissions                      12
Agricultural nitrous oxide emissions                12
CO2 emissions                                        0
CO2 emissions from bunker fuels                     60
CO2 emissions from cement production                60
CO2 emissions from electricity production (%)       60
CO2 emissions from fossil-fuels                     60
CO2 emissions from gas flaring                      60
CO2 emissions from gas fuel consumption             60
CO2 emissions from gaseous fuel consumption         36
CO2 emissions from gaseous fuel consumption (%)     36
CO2 emissi

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn import svm

In [ ]:
X = df.drop("MONTH_CLOSE",axis=1)
y = df["MONTH_CLOSE"]

train_X, test_X, train_y,  test_y = train_test_split(X, y, random_state=0)

In [ ]:
# Data imputation
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
train_X_imp = imp_mean.fit_transform(train_X)
test_X_imp = imp_mean.transform(test_X)

In [ ]:
# Data normalization
transformer = Normalizer()
train_X_trans = transformer.fit_transform(train_X_imp)
test_X_trans = transformer.transform(test_X_imp)

### SVM Regression

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'kernel':('linear', 'rbf','poly'), 'C':[0.01, 0.1, 1, 10]}
svr = svm.SVR()
clf = GridSearchCV(svr, parameters)
clf.fit(train_X_trans, train_y)
clf.score(test_X_trans, test_y)

-0.009145503948462919

### Polynomial Regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=3)
train_X_po = poly.fit_transform(train_X_trans)
test_X_po = poly.transform(test_X_trans)
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(train_X_po,train_y)
reg.score(test_X_po, test_y)

-8034697.741040067